# Custom Model

In [23]:
import tensorflow as tf

import keras

from keras import layers
from keras import models
from keras import optimizers

from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

import sklearn as sk




## Dataset Import

In [24]:
cifar10 = tf.keras.datasets.cifar10
(X_train, Y_train), (X_test,Y_test) = cifar10.load_data()



In [25]:
cmi = keras.Input(shape=(None,None,3))

end_models = []

for i in range(10):
    X_sample, Y_sample = sk.utils.resample(X_train, Y_train)

    #convert X_sample to float32
    X_sample = X_sample.astype('float32') / 255

    #convert Y_sample to one-hot encoding
    Y_sample = to_categorical(Y_sample)

    inpl = layers.Input(shape=(None,None,3))

    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inpl)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.2)(x)

    inner_model = models.Model(inpl, x)

    wrapper = models.Sequential()
    wrapper.add(layers.InputLayer(shape=(32,32,3)))
    wrapper.add(inner_model)
    wrapper.add(layers.Flatten())
    wrapper.add(layers.Dense(128, activation='relu'))
    wrapper.add(layers.Dense(10, activation='softmax'))

    wrapper.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    wrapper.fit(X_sample, Y_sample, epochs=10, batch_size=128, validation_split=0.05)

    inner_model.trainable = False

    end_models.append(inner_model)


Epoch 1/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.2921 - loss: 2.4540 - val_accuracy: 0.2536 - val_loss: 2.2829
Epoch 2/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 15s 40ms/step - accuracy: 0.4822 - loss: 1.5254 - val_accuracy: 0.5524 - val_loss: 1.3061
Epoch 3/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 15s 40ms/step - accuracy: 0.5746 - loss: 1.2164 - val_accuracy: 0.5472 - val_loss: 1.4384
Epoch 4/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 15s 41ms/step - accuracy: 0.6262 - loss: 1.0784 - val_accuracy: 0.6084 - val_loss: 1.1208
Epoch 5/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 18s 49ms/step - accuracy: 0.6419 - loss: 1.0542 - val_accuracy: 0.5804 - val_loss: 1.2789
Epoch 6/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 15s 40ms/step - accuracy: 0.6441 - loss: 1.0678 - val_accuracy: 0.6764 - val_loss: 1.0105
Epoch 7/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 15s 40ms/step - accuracy: 0.6517 - loss: 1.0662 - val_accuracy: 0.6100 - val_loss: 1.2014
Epoch 8/10
372/372 ━━━━━━━━━━━━━━━━━━━━ 15s 40ms/step - accuracy: 0.6599 - loss: 1.0457 - 

In [26]:
einp = layers.Input(shape=(32,32,3))

ensembles = []

for model in end_models:
    model.trainable = False

    x = model(einp)
    x = layers.Flatten()(x)

    ensembles.append(x)

x = layers.Concatenate()(ensembles)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(10, activation='softmax')(x)

ensemble = models.Model(einp, x)

print(ensemble.summary())

ensemble.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
ensemble.fit(X_train, to_categorical(Y_train), epochs=5, batch_size=128, validation_split=0.2)


print(ensemble.evaluate(X_test, to_categorical(Y_test)))


Model: "functional_84"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_46      │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_34       │ (None, 2, 2, 128) │    131,328 │ input_layer_46[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_39       │ (None, 2, 2, 128) │    131,328 │ input_layer_46[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_44       │ (None, 2, 2, 128) │    131,328 │ input_layer_46[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_49       │ (None, 2, 2, 128) │    131,328 │ input_layer_46[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_54       │ (None, 2, 2, 128) │    131,328 │ input_layer_46[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_59       │ (None, 2, 2, 128) │    131,328 │ input_layer_46[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_64       │ (None, 2, 2, 128) │    131,328 │ input_layer_46[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_69       │ (None, 2, 2, 128) │    131,328 │ input_layer_46[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_74       │ (None, 2, 2, 128) │    131,328 │ input_layer_46[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_79       │ (None, 2, 2, 128) │    131,328 │ input_layer_46[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_19          │ (None, 512)       │          0 │ functional_34[4]… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_20          │ (None, 512)       │          0 │ functional_39[4]… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_21          │ (None, 512)       │          0 │ functional_44[4]… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_22          │ (None, 512)       │          0 │ functional_49[4]… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_23          │ (None, 512)       │          0 │ functional_54[4]… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_24          │ (None, 512)       │          0 │ functional_59[4]

 Total params: 3,940,362 (15.03 MB)

 Trainable params: 2,627,082 (10.02 MB)

 Non-trainable params: 1,313,280 (5.01 MB)

None
Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 28s 68ms/step - accuracy: 0.3833 - loss: 2164.5784 - val_accuracy: 0.5602 - val_loss: 367.6675
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.5175 - loss: 532.6180 - val_accuracy: 0.6631 - val_loss: 211.9518
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 60ms/step - accuracy: 0.5637 - loss: 409.5088 - val_accuracy: 0.6723 - val_loss: 192.9414
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 18s 57ms/step - accuracy: 0.5905 - loss: 340.5475 - val_accuracy: 0.6305 - val_loss: 233.8991
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 17s 56ms/step - accuracy: 0.5871 - loss: 355.8476 - val_accuracy: 0.6701 - val_loss: 188.5928
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.6434 - loss: 218.1072
[220.29354858398438, 0.644599974155426]
